In [10]:
import pandas as pd
from elasticsearch5 import Elasticsearch
from elasticsearch5.client import IndicesClient

In [11]:
data_dir = '../category_names/data/'

In [12]:
df = pd.read_pickle(data_dir+'df_category_candidate.pkl')
df.head()

,category_names,cnt
0,기타,53
1,점퍼,51
2,스킨케어,49
3,기획세트,47
4,가방,45


In [4]:
es = Elasticsearch()
ic = IndicesClient(es)

In [5]:
def get_body(category):
    _body = {
        "tokenizer" : "nori_tokenizer",
        "filter" : ["lowercase"],
        "attributes" : ["leftPOS", "rightPOS"], 
        "explain" : True,
    }
    _body['text'] = category
    return _body

def get_terms(category):
    res = ic.analyze('nori_vanilla', body=get_body(category=category))
    return res['detail']['tokenizer']['tokens']

In [13]:
df['terms'] = df.category_names.apply(lambda category: get_terms(category))
df['len_terms'] = df.terms.apply(lambda x: len(x))

In [14]:
terms_list = []
for _, term_list in df['terms'].iteritems():
    temp = []
    for term_dict in term_list:
        temp.append(term_dict['token'])
    terms_list.append(" ".join(temp))
df['terms_str'] = terms_list

In [15]:
df_category = df.loc[df.len_terms != 1, ['category_names', 'terms_str']]
df_category = df_category.sort_values(by='category_names', ascending=True)

In [16]:
df_category.to_csv(data_dir+'category_candidate.csv', index=False)